# moralis_api

> Endpoints for the `moralis` api.

In [ ]:
#|default_exp moralis_api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from api_endpoints.etherscan_api import *
import time 
from moralis import evm_api
from openapi_evm_api.exceptions import ApiException
import json

In [ ]:
#| export

import os
from dotenv import load_dotenv
load_dotenv()
moralis_api_key = os.environ.get('moralis_api_key')

In [ ]:
#| export

import logging
#logging.getLogger().setLevel(logging.INFO)

logging.getLogger().setLevel(logging.WARNING)


In [ ]:
#| export

# def setup_price_cache(filename="../data/token_price_cache.json"):
#     """Initialize or reset the token price cache JSON file."""
#     empty_cache = {}
#     with open(filename, 'w') as f:
#         json.dump(empty_cache, f, indent=4)

# Uncomment the above function if you need to use it
# setup_price_cache()

class PriceFetcher:
    def __init__(self, etherscan_api_key, moralis_api_key, cache_filename="../data/token_price_cache.json"):
        self.cache_filename = cache_filename
        self.etherscan_api_key = etherscan_api_key
        self.moralis_api_key = moralis_api_key
        
        blockfetcher = BlockFetcher(self.etherscan_api_key)
        self.get_block_for_date = blockfetcher.get_block_for_date
        
        self.load_cache()

    def load_cache(self):
        try:
            with open(self.cache_filename, 'r') as f:
                self.cache = json.load(f)
        except FileNotFoundError:
            self.cache = {}

    def save_cache(self):
        with open(self.cache_filename, 'w') as f:
            json.dump(self.cache, f, indent=4)

    def get_token_price_on_date(self, date, tokenAddress):
        start_time = time.time()
        cache_key = f"{date}_{tokenAddress}"

        if cache_key in self.cache:
            cached_value = self.cache[cache_key]

            if cached_value == "API_ERROR":
                return None

            return cached_value

        pre_block_time = time.time()
        block_number = self.get_block_for_date(date)
        logging.info('Block fetching time: %s', time.time() - pre_block_time)

        if block_number is None:
            logging.info(f"Could not find block number for date {date}.")
            self.cache[cache_key] = "NO_DATA"
            self.save_cache()
            return None
        
        params = {
            "chain": "eth",
            "include": "percent_change",
            "address": tokenAddress,
            "to_block": float(block_number),
        }

        try:
            result = evm_api.token.get_token_price(
                api_key=self.moralis_api_key,
                params=params,
            )
            price = result['usdPrice']
            logging.info('API call time: %s', time.time() - pre_block_time)

            # Update the cache with the fetched price
            self.cache[cache_key] = price
            self.save_cache()  # Save cache after updating
            logging.info('Total function time: %s', time.time() - start_time)
            return price

        except ApiException:
            logging.error(f"ApiException occurred while fetching price for token {tokenAddress} on date {date}. It's possible the API did not find the token or encountered other issues.")
            self.cache[cache_key] = "API_ERROR"
            self.save_cache()  # Save cache after updating
            return None
        except Exception as e:
            logging.error(f"An unexpected error occurred while fetching price for token {tokenAddress} on date {date}: {str(e)}")
            self.cache[cache_key] = "NO_DATA"
            self.save_cache()
            return None


How to use `PriceFetcher`

In [ ]:
#| hide

#When cached, really fast
date = "2023-10-05"
tokenAddress = '0xae8627a4b3ec2c79c0c2971369482f004165a70b' #baconator, which `covalent` originally didn't succeed in getting `quote` for

price_fetcher = PriceFetcher(etherscan_api_key=etherscan_api_key, moralis_api_key=moralis_api_key)
price_fetcher.get_token_price_on_date(date=date, tokenAddress=tokenAddress)


9.745174815e-05

Alright, I guess we need to make this as fast as possible, or workout why it is slow....

ONE THING WE CAN DO is to get all the prices for a token at once.

In [ ]:
#| export
import datetime

def get_all_prices(tokenAddress:str,etherscan_api_key:str,moralis_api_key:str):
    "Get all prices for a token from inception to current date. So one price per day."

    #Alright. Let's get all the prices since inception for hpbitcoin.  
    #tokenAddress = '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9' #hpbitcoin
    price_fetcher = PriceFetcher(etherscan_api_key=etherscan_api_key, moralis_api_key=moralis_api_key)

    #Step 1): Get the creation date of the token.
    _creation_date = get_creation_date(contract_address=tokenAddress,api_key=etherscan_api_key)
    _creation_date = _creation_date.split(" ")[0]
    #print(f"Creation date: {_creation_date}")

    #Get all dates from then until now:
    dates_list = generate_dates_between(_creation_date, datetime.now().strftime("%Y-%m-%d"))
    #print(f"Number of dates: {len(dates_list)}")
    #print(f"First date: {dates_list[0]} | Last date: {dates_list[-1]}")

    for date in dates_list:
        price=price_fetcher.get_token_price_on_date(date=date, tokenAddress=tokenAddress)
        #print(f"The price of token on {date} was {price}")



In [ ]:
#| hide

# #dates = generate_dates_from_year(2014)

# import logging
# logging.getLogger().setLevel(logging.INFO)
# #logging.getLogger().setLevel(logging.WARNING)

# date = dates[-11]
# tokenAddress = '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9' #hpbitcoin

# price_fetcher = PriceFetcher(etherscan_api_key=etherscan_api_key, moralis_api_key=moralis_api_key)
# price_fetcher.get_token_price_on_date(date=date, tokenAddress=tokenAddress)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()